In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import networkx as nx
import random
import time
import pickle as cp
import sys
from tqdm import tqdm
import PrepareBatchGraph
import graph
import nstep_replay_mem
import nstep_replay_mem_prioritized
import mvc_env
import utils
import heapq
import scipy.linalg as linalg
import os
import pandas as pd

C:\Users\faraz\.conda\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [59]:
class Model(nn.Module):
    def __init__(self, embedding_size, w_initialization_std):
        super(Model, self).__init__()
        
        self.rand_generator = torch.normal
        self.embedding_size = embedding_size
        self.w_initialization_std = w_initialization_std
        # [2, embed_dim]
        self.w_n2l = nn.parameter.Parameter(data=self.rand_generator(0, self.w_initialization_std,\
                                                                     size=(2, self.embedding_size)))
        # [embed_dim, embed_dim]
        self.p_node_conv = nn.parameter.Parameter(data=self.rand_generator(0, self.w_initialization_std,\
                                                                           size=(self.embedding_size, self.embedding_size)))
        
    
            #p_node_conv = tf.Variable(tf.truncated_normal([self.embedding_size, self.embedding_size], stddev=initialization_stddev), tf.float32)

    def forward(self, node_input):

        x = torch.matmul(node_input, self.w_n2l)
        x = F.relu(x)

        return x


## Misc

In [51]:
x_a, y_a = torch.ones((100, 2)), torch.ones(100)
x_b, y_b = torch.zeros((100, 2)), torch.zeros(100)


In [65]:
for i in range (1):
    
    M = Model(1, 1)


    numb_parameters = 0
    for param in M.parameters():
        numb_parameters += np.prod(list(param.shape))
    print(numb_parameters)

    print(M(torch.ones((5, 2))))



2
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<ReluBackward0>)


In [53]:
optimizer = optim.SGD(M.parameters(), lr=0.01, momentum=0.9)
criterion = nn.BCEWithLogitsLoss()


In [54]:
for i in range(100):
    optimizer.zero_grad()
    # Forward pass, backward pass, and optimize
    outputs = M(x_a)
    loss = criterion(outputs[:,0], y_a.float())
    loss.backward()
    optimizer.step()
    
    optimizer.zero_grad()
    # Forward pass, backward pass, and optimize
    outputs = M(x_b)
    loss = criterion(outputs[:,0], y_b.float())
    loss.backward()
    optimizer.step()

In [55]:
M(torch.ones(20,2))

tensor([[3.2441],
        [3.2441],
        [3.2441],
        [3.2441],
        [3.2441],
        [3.2441],
        [3.2441],
        [3.2441],
        [3.2441],
        [3.2441],
        [3.2441],
        [3.2441],
        [3.2441],
        [3.2441],
        [3.2441],
        [3.2441],
        [3.2441],
        [3.2441],
        [3.2441],
        [3.2441]], grad_fn=<ReluBackward0>)

In [56]:
M(torch.zeros(20,2))

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<ReluBackward0>)